In [12]:
import pandas as pd
import torch
import torch.nn as nn
from torch.nn import functional as f

In [2]:
with open('paragraphs.txt' , 'r' , encoding='utf-8') as f:
    text = f.read()

In [3]:
print("length : ",len(text))

length :  8229768


In [4]:
print(text[:1000])

This biographical article related to French artistic gymnastics is a stub. You can help Wikipedia by expanding it.
InsideAR was the largest Augmented Reality event in Europe. It was organized and supported by metaio GmbH every year. The first event was held in 2010, had since expanded globally and was run at multiple locations around the world.  However, after Apple purchased metaio in May 2015, metaio cancelled the InsideAR conference 2015 without any statements about the conference's future. 
The Fearing Mind is an American horror television series that aired on the Fox Family Channel from October 21 until December 2, 2000. 
Bill Fearing, a famous writer of suspense thrillers, gets his ideas from things that happen in his family. When he gets an idea, the viewers enter his mind and see the gruesome events unfold. 
It had a bronze barrel and lacked a modern recoil system, using only an ineffective spring-mounted spade brake, and was virtually obsolescent on its introduction. Nonethele

In [5]:
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\]^_abcdefghijklmnopqrstuvwxyz{}
93


In [6]:
stringToNumbers = {ch:i for i,ch in enumerate(chars)}
numbersToString = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stringToNumbers[c] for c in s]
decode = lambda l: ''.join([numbersToString[i] for i in l])

print(encode("hey there , how are you"))
print(decode(encode("hey there , how are you")))

[72, 69, 89, 1, 84, 72, 69, 82, 69, 1, 13, 1, 72, 79, 87, 1, 65, 82, 69, 1, 89, 79, 85]
hey there , how are you


In [7]:
data=torch.tensor(encode(text),dtype=torch.long)
print(data[:1000])

tensor([53, 72, 73, 83,  1, 66, 73, 79, 71, 82, 65, 80, 72, 73, 67, 65, 76,  1,
        65, 82, 84, 73, 67, 76, 69,  1, 82, 69, 76, 65, 84, 69, 68,  1, 84, 79,
         1, 39, 82, 69, 78, 67, 72,  1, 65, 82, 84, 73, 83, 84, 73, 67,  1, 71,
        89, 77, 78, 65, 83, 84, 73, 67, 83,  1, 73, 83,  1, 65,  1, 83, 84, 85,
        66, 15,  1, 58, 79, 85,  1, 67, 65, 78,  1, 72, 69, 76, 80,  1, 56, 73,
        75, 73, 80, 69, 68, 73, 65,  1, 66, 89,  1, 69, 88, 80, 65, 78, 68, 73,
        78, 71,  1, 73, 84, 15,  0, 42, 78, 83, 73, 68, 69, 34, 51,  1, 87, 65,
        83,  1, 84, 72, 69,  1, 76, 65, 82, 71, 69, 83, 84,  1, 34, 85, 71, 77,
        69, 78, 84, 69, 68,  1, 51, 69, 65, 76, 73, 84, 89,  1, 69, 86, 69, 78,
        84,  1, 73, 78,  1, 38, 85, 82, 79, 80, 69, 15,  1, 42, 84,  1, 87, 65,
        83,  1, 79, 82, 71, 65, 78, 73, 90, 69, 68,  1, 65, 78, 68,  1, 83, 85,
        80, 80, 79, 82, 84, 69, 68,  1, 66, 89,  1, 77, 69, 84, 65, 73, 79,  1,
        40, 77, 66, 41,  1, 69, 86, 69, 

TRAINING data set

In [8]:
n=int(0.7*len(data))
train_data = data[:n]
# val_data = data[n:]

In [9]:
context_length=9
train_data[:context_length]

tensor([53, 72, 73, 83,  1, 66, 73, 79, 71])

In [11]:
torch.manual_seed(1337)
batch_size=5
block_size=10

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x=torch.stack([data[i:i+block_size] for i in ix])
    y=torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x,y

xb,yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):
  for t in range(block_size):
    context = xb[b, :t+1]
    target = yb[b,t]
    print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([5, 10])
tensor([[ 1, 72, 69, 76, 68,  1, 65, 84,  1, 84],
        [78, 68,  1, 67, 79, 78, 84, 73, 78, 85],
        [79, 76, 76, 85, 83, 75,  1, 73, 78,  1],
        [77, 65, 78,  1, 84, 79, 76, 68,  1, 43],
        [65, 79, 84, 73, 67,  1, 68, 85, 69,  1]])
targets:
torch.Size([5, 10])
tensor([[72, 69, 76, 68,  1, 65, 84,  1, 84, 72],
        [68,  1, 67, 79, 78, 84, 73, 78, 85, 69],
        [76, 76, 85, 83, 75,  1, 73, 78,  1, 84],
        [65, 78,  1, 84, 79, 76, 68,  1, 43, 65],
        [79, 84, 73, 67,  1, 68, 85, 69,  1, 84]])
----
when input is [1] the target: 72
when input is [1, 72] the target: 69
when input is [1, 72, 69] the target: 76
when input is [1, 72, 69, 76] the target: 68
when input is [1, 72, 69, 76, 68] the target: 1
when input is [1, 72, 69, 76, 68, 1] the target: 65
when input is [1, 72, 69, 76, 68, 1, 65] the target: 84
when input is [1, 72, 69, 76, 68, 1, 65, 84] the target: 1
when input is [1, 72, 69, 76, 68, 1, 65, 84, 1] the target: 84
wh

In [13]:
class BigramLanguageModel(nn.Module):

  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)

  def forward(self,index,targets):
    logits = self.token_embedding_table(index)
    return logits


  def generate(self,index,max_new_tokens):
    for _ in range(max_new_tokens):
      logits = self(index)
      logits = logits[:,-1,:]
      probs = f.softmax(logits,dim=-1)
      index_next = torch.multinomial(probs,num_samples=1)
      index = torch.cat((index,index_next),dim=1)

      return index
m=BigramLanguageModel(vocab_size)
out = m(xb,yb)
print(out.shape)

torch.Size([5, 10, 93])
